## WELCOME TO WEEK 6

The Epic Finale Week

And

# WELCOME TO THE **M**ODEL **C**ONTEXT **P**ROTOCOL!

And welcome back to OpenAI Agents SDK ❤️❤️❤️

### Please note

There may be changes here from the video as I'm always making updates!

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">To my Windows PC people - an important announcement</h2>
            <span style="color:#ff7800;">I have unpleasant news. There's a problem running MCP Servers on Windows PCs; Mac and Linux is fine. This is a known issue as of May 4th, 2025. I asked o3 with Deep Research to try to find workarounds; it <a href="https://chatgpt.com/share/6817bbc3-3d0c-8012-9b51-631842470628">confirmed the issue</a> and confirmed the workaround.<br/><br/>
            The workaround is a bit of a bore. It is to take advantage of "WSL", the Microsoft approach for running Linux on your PC. You'll need to carry out more setup instructions! But it's quick, and several students have confirmed that this works perfectly for them, then the Week 6 MCP labs work. Plus, WSL is actually a great way to build software on your Windows PC.<br/>
            The WSL Setup instructions are in the Setup folder, <a href="../setup/SETUP-WSL.md">in the file called SETUP-WSL.md here</a>. I do hope this only holds you up briefly - you should be back up and running quickly. Oh the joys of working with bleeding-edge technology!<br/><br/>
            With many thanks to students Markus, Abhi, Hui-Ling, and several others, for helping me work on it and confirming the fix.
            </span>
        </td>
    </tr>
</table>

In [6]:
# The imports

# If you get "ModuleNotFoundError: No module named 'agents'", 
# you need to install the openai-agents package.
# Run this in your terminal (not in the notebook):
#   cd /Users/oluwatobilobasokoya/AI/agents
#   uv sync
# Or if that doesn't work, try:
#   uv pip install openai-agents
# Then restart your Jupyter kernel and try again.

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [7]:
load_dotenv(override=True)

True

### Let's use MCP in OpenAI Agents SDK

1. Create a Client

2. Have it spawn a server

3. Collect the tools that the server can use

Let's try the Fetch mcp-server that we looked at last week

### Special note - in case you updated the OpenAI Agents framework:

OpenAI Agents SDK recently upgraded their SDK and made a breaking change (thank you, OpenAI 😂). I haven't yet updated the version in this repo to the latest version. If you've upgraded the package yourself, then you might get an error about missing positional arguments. If you get that error, you'll need to replace `await server.list_tools()` with `await server.session.list_tools()` and change the last line from `fetch_tools` to `fetch_tools.tools`. And you'll need to make this change every time you get this error! The joys of working at the bleeding edge..

In [8]:
fetch_params = {"command": "uvx", "args":["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=30) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

## Extra installation step - if you don't have Node and Playwright on your computer

The next MCP tool uses node (the Javascript Server), and it needs you to have the command 'npx' installed on your computer.

Extra setup steps are here:

[Node and Playwright setup](../setup/SETUP-node.md)

## And now 2 more MCP servers - this time Javascript based, with node

### If the next cell hangs, see the important troubleshooting at the end of the SETUP-node instructions linked above.

In [9]:
import os
from agents.mcp.server import MCPServerStdio
import logging
logging.basicConfig(level=logging.DEBUG)

playwright_params = {
    "command": "/usr/local/bin/npx",
    "args": ["-y", "@playwright/mcp@latest"],
    "env": {
        **os.environ,
        "NODE_ENV": "production",
        "PATH": os.environ.get("PATH", "")
    }
}

async def list_playwright_tools():
    try:
        async with MCPServerStdio(
            params=playwright_params,
            client_session_timeout_seconds=120
        ) as server:
            print("Server connected successfully")
            tools = await server.list_tools()
            print(f"Retrieved {len(tools)} tools")
            print(tools)
            return tools
    except Exception as e:
        import traceback
        error_msg = f"Error: {e}\n{traceback.format_exc()}"
        print(error_msg)
        return error_msg

# Run it
tools = await list_playwright_tools()

Server connected successfully
Retrieved 22 tools
[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None, execution=None), Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, 

In [10]:
sandbox_path = os.path.abspath(os.path.join(os.getcwd(),"sandbox"))
files_params = {"command":"/usr/local/bin/npx", "args":["-y","@modelcontextprotocol/server-filesystem", sandbox_path]}

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as server:
    file_tools = await server.list_tools()
file_tools

[Tool(name='read_file', title='Read File (Deprecated)', description='Read the complete contents of a file as text. DEPRECATED: Use read_text_file instead.', inputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'path': {'type': 'string'}, 'tail': {'description': 'If provided, returns only the last N lines of the file', 'type': 'number'}, 'head': {'description': 'If provided, returns only the first N lines of the file', 'type': 'number'}}, 'required': ['path']}, outputSchema={'$schema': 'http://json-schema.org/draft-07/schema#', 'type': 'object', 'properties': {'content': {'type': 'string'}}, 'required': ['content'], 'additionalProperties': False}, icons=None, annotations=ToolAnnotations(title=None, readOnlyHint=True, destructiveHint=None, idempotentHint=None, openWorldHint=None), meta=None, execution=ToolExecution(taskSupport='forbidden')),
 Tool(name='read_text_file', title='Read Text File', description="Read the complete contents of a fil

### And now.. bring on the Agent with Tools!

In [12]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task,
including accepting all cookies and clicking 'not now' as 
appropriate to get to the content you need. If one website isn't fruitful, try another.
Be persistent until you have solved your assignment,
trying different options and sites as needed.

IMPORTANT: All files must be saved in the 'sandbox' directory. 
For example, save files as 'sandbox/filename.md', not just 'filename.md'.
"""

async with MCPServerStdio(params=files_params, client_session_timeout_seconds=30) as mcp_server_files:
    async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=30) as mcp_server_browser:
        agent = Agent(
            name= "investigator",
            instructions=instructions,
            model= "gpt-4o-mini",
            mcp_servers=[mcp_server_files, mcp_server_browser]
        )

        with trace("investigate"):
            result = await Runner.run(agent, "Find a great recipe for Banoffee Pie, then summarize it in markdown to banoffee.md")
            print(result.final_output)



DEBUG:openai.agents:Creating trace investigate with id trace_23e4af82f3cc4b5386ca57c3b710d548
DEBUG:openai.agents:Setting current trace: trace_23e4af82f3cc4b5386ca57c3b710d548
DEBUG:openai.agents:Creating span <agents.tracing.span_data.MCPListToolsSpanData object at 0x11443dbd0> with id None
DEBUG:openai.agents:Creating span <agents.tracing.span_data.MCPListToolsSpanData object at 0x11599e410> with id None
DEBUG:openai.agents:Creating span <agents.tracing.span_data.AgentSpanData object at 0x115b23de0> with id None
DEBUG:openai.agents:Running agent investigator (turn 1)
DEBUG:openai.agents:Creating span <agents.tracing.span_data.ResponseSpanData object at 0x1153e0dd0> with id None
DEBUG:openai.agents:Calling LLM
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/responses', 'headers': {'User-Agent': 'Agents/Python 0.6.4'}, 'files': None, 'idempotency_key': 'stainless-python-retry-d7f06a22-527d-46f3-b623-f47d185416a5', 'json_data': {'include': [], 'input': [{'content'

I've found a great recipe for Banoffee Pie and summarized it in markdown format. You can find the recipe in [banoffee.md](sandbox/banoffee.md).

Here's a brief overview:

# Banoffee Pie Recipe

## Ingredients
- **4** bananas, sliced
- **394g** caramel or dulce de leche
- **300ml** double cream
- Dark chocolate (optional)

### For the pastry
- **100g** butter, chilled (plus extra for greasing)
- **200g** plain flour
- **1** medium egg, separated
- **1 tbsp** golden caster sugar

## Method
1. **Make the pastry case**: Blend butter and flour until crumb-like. Add egg yolk and sugar; mix. Gradually add cold water until dough forms.
2. Knead gently, wrap in cling film, and chill for **30 minutes**.
3. Preheat oven to **190C/170C fan/gas 4**. Prepare the tart tin with rolled-out pastry.
4. Bake for **15 minutes** with parchment and weights. Brush with egg white and bake for another **15-20 minutes** until golden. Cool completely.
5. Layer half the caramel, bananas, and remaining caramel in t

### Check out the trace

https://platform.openai.com/traces

### Now take a look at some MCP marketplaces

https://mcp.so

https://glama.ai/mcp

https://smithery.ai/

https://huggingface.co/blog/LLMhacker/top-11-essential-mcp-libraries

HuggingFace great community article:
https://huggingface.co/blog/Kseniase/mcp



